# Data Preparation

## Download Transportation Network

In [1]:
import osmnx as ox

G = ox.graph_from_place("Bangkok, Thailand", network_type='drive')

## Prepare Network for Infomap Clustering

In [2]:
import networkx as nx
# Reassign node ids to make it's compatible with infomap
original_to_new = {node: idx for idx, node in enumerate(G.nodes())}
new_to_original = {idx: node for node, idx in original_to_new.items()}

G_relabelled = nx.relabel_nodes(G, original_to_new)

In [3]:
# Verify the edges are preserved
print("Original Graph Edges (first 5):", list(G.edges(data=True))[:5])
print("Relabelled Graph Edges (first 5):", list(G_relabelled.edges(data=True))[:5])

Original Graph Edges (first 5): [(51479828, 1337580877, {'osmid': 202235939, 'highway': 'tertiary', 'maxspeed': '50', 'name': 'ถนนบรรทัดทอง', 'oneway': False, 'reversed': False, 'length': np.float64(61.173863186207235)}), (51479828, 1337580894, {'osmid': 202235939, 'highway': 'tertiary', 'maxspeed': '50', 'name': 'ถนนบรรทัดทอง', 'oneway': False, 'reversed': True, 'length': np.float64(79.09113824839476)}), (51479828, 5933302530, {'osmid': 337523545, 'highway': 'residential', 'name': 'ซอยพญานาค', 'oneway': True, 'reversed': False, 'length': np.float64(105.3227138366798), 'geometry': <LINESTRING (100.526 13.753, 100.526 13.753, 100.526 13.753, 100.525 13.753)>}), (51479831, 1337580894, {'osmid': 202235939, 'highway': 'tertiary', 'maxspeed': '50', 'name': 'ถนนบรรทัดทอง', 'oneway': False, 'reversed': False, 'length': np.float64(2.3938952688706316)}), (51479831, 5933302557, {'osmid': 118986832, 'highway': 'residential', 'name': 'ซอยโรงแรมเจริญผล', 'oneway': False, 'reversed': False, 'length'

## Simulate Traffic Density

- Create an agent-based model where agents are assigned random origin-destination (OD) pairs.
- Assign traffic density to edges based on the number of agents traversing them.

In [11]:
import random

for edge in G_relabelled.edges:
    G_relabelled.edges[edge]['traffic_density'] = random.randint(1, 1000000)

# Identify Homogeneous, Connected, and Compact Sub-Regions

1. Graph Representation
- Convert the network into a weighted graph where nodes are intersections and edges are roads.
- Use traffic density as edge weights.
  
2. Infomap Clustering
- Use the Infomap algorithm to find homogeneous and connected clusters.

In [12]:
from infomap import Infomap
import numpy as np

infomap = Infomap()

for u, v, data in G_relabelled.edges(data=True):
    
    weight = float(data.get('traffic_density', 1))
    infomap.addLink(np.uint32(u), np.uint32(v), weight)
    
infomap.run()

. Found 11 levels with codelength 3.486065313

=> Trial 1/1 finished in 9.65361933s with codelength 3.48606531


Summary after 1 trial
Best end modular solution in 11 levels:
Per level number of modules:         [          2,          23,         149,         918,        4846,       17431,       21191,        9535,        1522,         124,           0] (sum: 55741)
Per level number of leaf nodes:      [          0,           0,           0,           0,           0,        4973,       53129,       77030,       36249,        5504,         493] (sum: 177378)
Per level average child degree:      [          2,        11.5,     6.47826,     6.16107,     5.27887,     4.62319,     4.26367,     4.08499,      3.9613,     3.69777,     3.97581] (average: 4.19929)
Per level codelength for modules:    [0.000055693, 0.001428211, 0.004676260, 0.015965020, 0.066907521, 0.235207270, 0.270990388, 0.112790383, 0.015697833, 0.001225053, 0.000000000] (sum: 0.724943632)
Per level codelength for leaf nodes:

In [13]:
# Extract clusters
communities = {new_to_original[node_id]: module_id for node_id, module_id in infomap.modules}

# View number of clusters
set(communities.values())

{1, 2, 3}

In [14]:
# Add community information to nodes
nx.set_node_attributes(G, communities, name="community")

# Add community information to edges
for u, v, data in G.edges(data=True):
    data["community"] = communities.get(u)

In [15]:
# Convert nodes to GeoJSON format
node_features = []
for node, data in G.nodes(data=True):
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [data["x"], data["y"]] 
        },
        "properties": {
            "id": node,
            "community": data["community"],  
        },
    }
    node_features.append(feature)

nodes_geojson = {
    "type": "FeatureCollection",
    "features": node_features,
}

# Convert edges to GeoJSON format
edge_features = []
for u, v, data in G.edges(data=True):
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "LineString",
            "coordinates": [
                [G.nodes[u]["x"], G.nodes[u]["y"]],  
                [G.nodes[v]["x"], G.nodes[v]["y"]],  
            ],
        },
        "properties": {
            "source": u,
            "target": v,
            "community": data["community"],  
        },
    }
    edge_features.append(feature)

edges_geojson = {
    "type": "FeatureCollection",
    "features": edge_features,
}

In [16]:
import json

with open("nodes_with_communities.geojson", "w") as f:
    json.dump(nodes_geojson, f)

with open("edges_with_communities.geojson", "w") as f:
    json.dump(edges_geojson, f)

print("GeoJSON files saved!")

GeoJSON files saved!
